# Capítulo 7 - Pré Processamento de Dados

In [54]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import janitor as jn
from sklearn.experimental import (
    enable_iterative_imputer,
)
from sklearn import (
    ensemble,
    impute,
    model_selection,    
    preprocessing,
    tree,
)

In [4]:
url = ("https://hbiostat.org/data/repo/titanic3.csv")
df = pd.read_csv(url)

## Padronização de Dados

In [5]:
X2 = pd.DataFrame(
    {
        "a": range(5),
        "b": [-100, -50, 0, 200, 1000]
    }
)
X2

,a,b
0,0,-100
1,1,-50
2,2,0
3,3,200
4,4,1000


O `sklearn` possuí um método `.fit_transform()` que combina `.fit()` e `.transform()` em um único método para facilitar o pré-processamento de dados.

O método `fit` é usado para calcular os parâmetros de ajuste de um modelo, enquanto o método `transform` é usado para aplicar os parâmetros de ajuste em um modelo.

Os parâmetros de ajuste são usados para padronizar os dados de entrada.

In [6]:
std = preprocessing.StandardScaler()
std.fit_transform(X2)

array([[-1.41421356, -0.75995002],
       [-0.70710678, -0.63737744],
       [ 0.        , -0.51480485],
       [ 0.70710678, -0.02451452],
       [ 1.41421356,  1.93664683]])

In [9]:
print(f'O desvio padrão é {std.scale_}\nA média é {std.mean_}\ns valores são {std.var_}')

O desvio padrão é [  1.41421356 407.92156109]
A média é [  2. 210.]
s valores são [2.000e+00 1.664e+05]


É possível padronizar usando `pandas`

In [10]:
X_std = (X2 - X2.mean())/ X2.std()
X_std

,a,b
0,-1.264911,-0.679720
1,-0.632456,-0.570088
2,0.000000,-0.460455
3,0.632456,-0.021926
4,1.264911,1.732190


In [11]:
X_std.mean()

a    4.440892e-17
b    0.000000e+00
dtype: float64

In [12]:
X_std.std()

a    1.0
b    1.0
dtype: float64

É possível utilizar a biblioteca `fastai` para padronizar os dados.

In [18]:
from fastai.tabular.all import *

In [36]:
X3 = X2.copy()

count_names = ['a', 'b']

dls = TabularDataLoaders.from_df(X3, procs=[Normalize], cont_names=count_names)
dls.train_ds.conts

,a,b
2,-0.447214,-0.681500
3,0.447214,-0.207413
4,1.341641,1.688935
1,-1.341641,-0.800022


## Escalonamento de Dados

É possível utilizar o método `MinMaxScaler` para escalonar os dados.

In [37]:
from sklearn import preprocessing

In [38]:
mms = preprocessing.MinMaxScaler()
mms.fit_transform(X2)

array([[0.        , 0.        ],
       [0.25      , 0.04545455],
       [0.5       , 0.09090909],
       [0.75      , 0.27272727],
       [1.        , 1.        ]])

Também é possível escaloar os dados utilizando `pandas`.

In [43]:
(X2 - X2.min()) / (X2.max() - X2.min())

,a,b
0,0.00,0.000000
1,0.25,0.045455
2,0.50,0.090909
3,0.75,0.272727
4,1.00,1.000000


## Variáveis Dummy

In [44]:
X_cat = pd.DataFrame(
    {
        "nome": ["Paulo", "Luana"],
        "pais": ["Alemanha", "Brasil"],
    }
)
X_cat

,nome,pais
0,Paulo,Alemanha
1,Luana,Brasil


In [45]:
pd.get_dummies(X_cat)

,nome_Luana,nome_Paulo,pais_Alemanha,pais_Brasil
0,False,True,True,False
1,True,False,False,True


In [46]:
pd.get_dummies(X_cat, drop_first=True) # drop_first=True para evitar a multicolinearidade

,nome_Paulo,pais_Brasil
0,True,False
1,False,True


Também é possível utilizar `pyjanitor` para criar variáveis dummy.

In [48]:
X_cat2 = pd.DataFrame(
    {
        "A": [1, 2, None],
        "nome": ["Paulo", "Luana", "João"],
    }
)
X_cat2

,A,nome
0,1.0,Paulo
1,2.0,Luana
2,NaN,João


In [55]:
jn.expand_column(X_cat2, "nome", sep=",")

,A,nome,João,Luana,Paulo
0,1.0,Paulo,0,0,1
1,2.0,Luana,0,1,0
2,NaN,João,1,0,0


## Codificador de Rótulos

In [56]:
from sklearn import preprocessing

In [59]:
lab = preprocessing.LabelEncoder()
lab.fit_transform(X_cat.nome)

array([1, 0])

Para decodificar os rótulos, é possível utilizar o método `inverse_transform()`.

In [71]:
lab.inverse_transform([1,0])

array(['Paulo', 'Luana'], dtype=object)

O `pandas` também possui um método para codificar rótulos.

Deve-se converter a coluna em um tipo de dados categórico antes de codificar os rótulos, em seguida extrair o código númerico.

Dessa forma, cria-se uma nova coluna com os rótulos codificados. Aconselhavel o uso do `.as_ordered()` para manter a ordem dos rótulos.

In [74]:
y = X_cat.nome.astype(
    "category"
).cat.as_ordered().cat.codes # cat.codes é o mesmo que LabelEncoder

In [82]:
print(f'X_cat\n{X_cat}\n\ny\n{y}')

X_cat
    nome      pais
0  Paulo  Alemanha
1  Luana    Brasil

y
0    1
1    0
dtype: int8
